## Test how hard one can infere with the VarEM Alogrithm


In [37]:
from models.ICA_EM import *
from models.dgp import *
from models.metrics import *
import importlib, sys
importlib.reload(sys.modules['models.ICA_EM'])
importlib.reload(sys.modules['models.dgp'])
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [38]:
class CausalVarEM(VarEM):
    def __init__(self, update_sigma=False, true_A=None, tol=1e-4, mode = "init", **kwargs):
        if mode not in ["init", "each"]:
            raise ValueError("mode must be either 'init' or 'each'")
        self.mode = mode
        super().__init__(update_sigma=update_sigma, true_A=true_A, tol=tol, **kwargs)
    
    def update_A(self): # we can force causal structure 
        temp1 = np.zeros((self.I, self.J))
        temp2 = np.zeros((self.J, self.J))

        for i in range(self.n):
            omega_i = self._omega_mat(i)
            M_i = self._M_mat(omega_i)
            x_outer = np.outer(self.X[i,:], self.X[i,:])
            temp1 += x_outer @ M_i.T @ self.A @ omega_i
            temp2 += omega_i @ (np.eye(self.J) - self.A.T @ M_i @ (np.eye(self.I) - x_outer @ M_i.T) @ self.A @ omega_i)
            self._update_xi(i, M_i, omega_i, x_outer)
        A_new = temp1 @ np.linalg.inv(temp2)
        if self.update_sigma:
            self._update_sigma(A_new)


        # calculate the difference between the old and new A
        diff = np.linalg.norm(self.A - A_new, ord='fro')
        self.A = A_new
        if self.mode == "each":
            self._enforce_causal_structure()
        return diff
    
    def _initilize_A(self):
        super()._initilize_A()
        self._enforce_causal_structure()
        
    def _enforce_causal_structure(self):
        self.A[0,1] = 0
        # set ones
        self.A[0,0] = 1
        self.A[1,1] = 1
        # set controls to 1
        for j in range(2, self.J-1):
            self.A[j,j+1] = 1
            



## simulation to test if we should enforce causal strucutre each time


In [41]:
n = 10000
J = 6
I = 5
f_score_init = []
f_score_each = []
mean_squared_error_init = []
mean_squared_error_each = []
ll_score_init = []
ll_score_each = []
data = dgp(prior={"loc" : 0, "scale" : 1}, noise_dict=  {"loc" : 0, "scale" : 1})
for i in tqdm.tqdm(range(100)):
    data.generate_data(n=n, J=J, I=I, random_state=i)
    CausalVarEM_est = CausalVarEM(update_sigma=False,   true_A= data.mixing_matrix_observed, tol=1e-4, max_iter=200, random_seed=1)
    CausalVarEM_est.fit(data.data_observed, J = J,
                      noise_params= {"mean" : 0, "std" : 1}, progress_bar= False)
    best_perm, score = f_score(data.mixing_matrix_observed, CausalVarEM_est.A)
    f_score_init.append(score)
    singals_estimation_VAR = CausalVarEM_est.Signals[:,best_perm]
    mean_squared_error_init.append(mean_squared_error(data.signals, singals_estimation_VAR))
    ll_score_init.append(likelihood_score(data.signals, singals_estimation_VAR, normalize=True))

    CausalVarEM_est = CausalVarEM(update_sigma=False,   true_A= data.mixing_matrix_observed, tol=1e-4, max_iter=200, random_seed=1, mode = "each")
    CausalVarEM_est.fit(data.data_observed, J = J,
                      noise_params= {"mean" : 0, "std" : 1}, progress_bar= False)
    best_perm, score = f_score(data.mixing_matrix_observed, CausalVarEM_est.A)
    f_score_each.append(score)
    singals_estimation_VAR = CausalVarEM_est.Signals[:,best_perm]
    mean_squared_error_each.append(mean_squared_error(data.signals, singals_estimation_VAR))
    ll_score_each.append(likelihood_score(data.signals, singals_estimation_VAR, normalize=True))

df = pd.DataFrame({"f_score_init" : f_score_init, 
                   "f_score_each" : f_score_each, 
                   "mean_squared_error_init" : mean_squared_error_init, 
                   "mean_squared_error_each" : mean_squared_error_each,
                     "ll_score_init" : ll_score_init, 
                     "ll_score_each" : ll_score_each})
df.to_csv("results/causal_var_em.csv")


  0%|          | 0/100 [00:02<?, ?it/s]


KeyboardInterrupt: 